In [ ]:
# 1. Imports & Settings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("default")
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")
sns.set_theme()


## 2. Load the Data

We load the three CSV files provided:

- `sales_data.csv`
- `product_info.csv`
- `customer_info.csv`


In [ ]:
# 2.1 Read CSVs

sales_df = pd.read_csv("sales_data.csv")
product_df = pd.read_csv("product_info.csv")
customer_df = pd.read_csv("customer_info.csv")

sales_df.head()


## 3. Data Cleaning

We clean each DataFrame separately:

- Strip whitespace
- Standardise casing
- Convert dates
- Handle missing values
- Remove duplicates
- Validate numeric columns


In [ ]:
# 3.1 Standardise text columns

# 3.1 Standardise text columns

# Sales: delivery_status, payment_method, region
sales_df['delivery_status'] = sales_df['delivery_status'].str.strip().str.title()
sales_df['payment_method'] = sales_df['payment_method'].str.strip().str.title()
sales_df['region'] = sales_df['region'].str.strip().str.title()

# Product: product_name, category, supplier_code
product_df['product_name'] = product_df['product_name'].str.strip().str.title()
product_df['category'] = product_df['category'].str.strip().str.title()
product_df['supplier_code'] = product_df['supplier_code'].str.strip().str.upper()

# Customer: gender, region, loyalty_tier
customer_df['gender'] = customer_df['gender'].str.strip().str.title()
customer_df['region'] = customer_df['region'].str.strip().str.title()

customer_df['loyalty_tier'] = (
    customer_df['loyalty_tier']
    .astype(str)
    .str.strip()
    .str.lower()
    .map({"bronze": "Bronze", "silver": "Silver", "gold": "Gold"})
)


# 3.2 Convert dates

sales_df["order_date"] = pd.to_datetime(sales_df["order_date"], errors="coerce")
customer_df["signup_date"] = pd.to_datetime(customer_df["signup_date"], errors="coerce")
product_df["launch_date"] = pd.to_datetime(product_df["launch_date"], errors="coerce")


# 3.3 Missing values overview

print("Sales missing:\n", sales_df.isnull().sum())
print("\nProduct missing:\n", product_df.isnull().sum())
print("\nCustomer missing:\n", customer_df.isnull().sum())


# Simple filling strategy (as suggested in brief)

# --- Sales Data ---

# discount_applied: fill missing with 0.0
if "discount_applied" in sales_df.columns:
    sales_df["discount_applied"] = sales_df["discount_applied"].fillna(0.0)

# Categorical: fill with 'Unknown'
for col in ["delivery_status", "payment_method", "region"]:
    if col in sales_df.columns:
        sales_df[col] = sales_df[col].fillna("Unknown")



# --- Product Data ---

# Categorical: fill with 'Unknown'
for col in ["category", "supplier_code"]:
    if col in product_df.columns:
        product_df[col] = product_df[col].fillna("Unknown")

# base_price: fill missing with 0
if "base_price" in product_df.columns:
    product_df["base_price"] = product_df["base_price"].fillna(0)

# launch_date: convert and let missing become NaT
if "launch_date" in product_df.columns:
    product_df["launch_date"] = pd.to_datetime(product_df["launch_date"], errors="coerce")



# --- Customer Data ---

# Categorical: fill with 'Unknown'
for col in ["gender", "region", "loyalty_tier"]:
    if col in customer_df.columns:
        customer_df[col] = customer_df[col].fillna("Unknown")

# signup_date: convert and let missing become NaT
if "signup_date" in customer_df.columns:
    customer_df["signup_date"] = pd.to_datetime(customer_df["signup_date"], errors="coerce")


# 3.4 Remove duplicates

sales_df = sales_df.drop_duplicates(subset="order_id")
product_df = product_df.drop_duplicates(subset="product_id")
customer_df = customer_df.drop_duplicates(subset="customer_id")

# 3.5 Validate numeric columns (no negative values)

# --- Sales numeric columns ---
numeric_cols_sales = ["quantity", "unit_price", "discount_applied"]

for col in numeric_cols_sales:
    if col in sales_df.columns:
        sales_df = sales_df[sales_df[col] >= 0]


# --- Product numeric columns ---
numeric_cols_product = ["base_price"]

for col in numeric_cols_product:
    if col in product_df.columns:
        product_df = product_df[product_df[col] >= 0]


# --- Customer numeric columns (none in this dataset) ---





## 4. Merge the Data

We merge:

1. `sales_df` with `product_df` on `product_id`  
2. Then with `customer_df` on `customer_id`  

We use a left join to keep all sales transactions.


In [ ]:
merged_df = (
    sales_df
    .merge(product_df, on="product_id", how="left", suffixes=("", "_prod"))
    .merge(customer_df, on="customer_id", how="left", suffixes=("", "_cust"))
)

merged_df.info()
merged_df.head()


## 5. Feature Engineering

We create:

- `revenue = quantity × unit_price × (1 - discount_applied)`
- `order_week` (ISO week number)
- `price_band` (Low / Medium / High)
- `days_to_order` (days between product launch and order)
- `email_domain` (e.g. gmail.com)
- `is_late` (True if delivery_status = "Delayed")


In [ ]:
# 5.1 Revenue

merged_df["revenue"] = (
    merged_df["quantity"] * merged_df["unit_price"] * (1 - merged_df["discount_applied"])
)

# 5.2 Order week (ISO week number)

merged_df["order_week"] = merged_df["order_date"].dt.isocalendar().week

# 5.3 Price band

bins = [0, 15, 30, np.inf]
labels = ["Low", "Medium", "High"]

merged_df["price_band"] = pd.cut(
    merged_df["unit_price"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# 5.4 Days from launch to order

merged_df["days_to_order"] = (
    merged_df["order_date"] - merged_df["launch_date"]
).dt.days

# 5.5 Email domain

merged_df["email_domain"] = merged_df["email"].str.split("@").str[-1]

# 5.6 Is late? (delivery_status == "Delayed")

merged_df["is_late"] = merged_df["delivery_status"] == "Delayed"


Summary Tables

In [ ]:
# Weekly Revenue trends by region

weekly_region_rev = (
    merged_df
    .groupby(["order_week", "region"])["revenue"]
    .sum()
    .reset_index()
)

weekly_region_rev.head()

# Product Category Performance

category_perf = (
    merged_df
    .groupby("category")
    .agg(
        total_revenue=("revenue", "sum"),
        total_quantity=("quantity", "sum"),
        avg_discount=("discount_applied", "mean")
    )
    .reset_index()
    .sort_values("total_revenue", ascending=False)
)

category_perf.head()



# Customer behaviour by loyalty tier and signup month

merged_df["signup_month"] = merged_df["signup_date"].dt.to_period("M").astype(str)

loyalty_behaviour = (
    merged_df
    .groupby(["loyalty_tier", "signup_month"])
    .agg(
        total_revenue=("revenue", "sum"),
        orders=("order_id", "nunique"),
        customers=("customer_id", "nunique")
    )
    .reset_index()
)

loyalty_behaviour.head()


# Delivery performance by region and price_band

delivery_performance = (
    merged_df
    .groupby(["region", "price_band"])
    .agg(
        total_orders=("order_id", "nunique"),
        late_deliveries=("is_late", "sum"),
        late_rate=("is_late", "mean")
    )
    .reset_index()
)

delivery_performance.head()


# Preferred payment methods by loyalty tier

payment_pref = (
    merged_df
    .groupby(["loyalty_tier", "payment_method"])
    .agg(
        total_orders=("order_id", "nunique"),
        revenue=("revenue", "sum")
    )
    .reset_index()
    .sort_values(["loyalty_tier", "total_orders"], ascending=[True, False])
)

payment_pref.head()


Visualisations

In [ ]:
# Line Plot - Weekly revenue trends by region

plt.figure(figsize=(10, 6))
sns.lineplot(
    data=weekly_region_rev,
    x="order_week",
    y="revenue",
    hue="region",
    marker="o"
)
plt.title("Weekly Revenue by Region")
plt.xlabel("ISO Week")
plt.ylabel("Revenue (£)")
plt.tight_layout()
plt.show()

# Bar Chart - Top 5 categories by revenue

top5_cat = category_perf.nlargest(5, "total_revenue")

plt.figure(figsize=(8, 5))
sns.barplot(data=top5_cat, x="category", y="total_revenue")
plt.title("Top 5 Categories by Revenue")
plt.xlabel("Category")
plt.ylabel("Revenue (£)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Boxplot - quantity vs discount across categories

plt.figure(figsize=(10, 6))
sns.boxplot(
    data=merged_df,
    x="category",
    y="discount_applied"
)

plt.title("Discount Applied Across Product Categories")
plt.xlabel("Product Category")
plt.ylabel("Discount Applied")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# Heatmap - correlation between revenue, discount, and quantity

corr_cols = ["revenue", "discount_applied", "quantity"]
corr_matrix = merged_df[corr_cols].corr()

plt.figure(figsize=(6, 4))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap="coolwarm"
)

plt.title("Correlation Between Revenue, Discount, and Quantity")
plt.tight_layout()
plt.show()


# Countplot - orders by loyalty tier (with hue = region)

plt.figure(figsize=(10, 6))
sns.countplot(
    data=merged_df,
    x="loyalty_tier",
    hue="region"
)

plt.title("Orders by Loyalty Tier and Region")
plt.xlabel("Loyalty Tier")
plt.ylabel("Number of Orders")
plt.tight_layout()
plt.show()


# Stacked bar chart - delivery status by price band

# Create a crosstab
delivery_crosstab = pd.crosstab(
    merged_df["price_band"],
    merged_df["delivery_status"]
)

# Plot
delivery_crosstab.plot(
    kind="bar",
    stacked=True,
    figsize=(10, 6)
)

plt.title("Delivery Status by Price Band")
plt.xlabel("Price Band")
plt.ylabel("Number of Orders")
plt.legend(title="Delivery Status")
plt.tight_layout()
plt.show()
